In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/apple-disease-detection/apple_disease_classification/Untitled.ipynb
/kaggle/input/apple-disease-detection/apple_disease_classification/.ipynb_checkpoints/Untitled-checkpoint.ipynb
/kaggle/input/apple-disease-detection/apple_disease_classification/Test/Blotch_Apple/149.jpg
/kaggle/input/apple-disease-detection/apple_disease_classification/Test/Blotch_Apple/143.jpg
/kaggle/input/apple-disease-detection/apple_disease_classification/Test/Blotch_Apple/131.jpg
/kaggle/input/apple-disease-detection/apple_disease_classification/Test/Blotch_Apple/130.jpg
/kaggle/input/apple-disease-detection/apple_disease_classification/Test/Blotch_Apple/120.jpg
/kaggle/input/apple-disease-detection/apple_disease_classification/Test/Blotch_Apple/124.jpg
/kaggle/input/apple-disease-detection/apple_disease_classification/Test/Blotch_Apple/123.jpg
/kaggle/input/apple-disease-detection/apple_disease_classification/Test/Blotch_Apple/137.jpg
/kaggle/input/apple-disease-detection/apple_disease_classifica

In [26]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [27]:
data = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/apple-disease-detection/apple_disease_classification/Train',
    image_size=(128, 128),
    batch_size=16,
    validation_split=0.2,  # Split the dataset into training and validation
    subset="training",      # Specify "training" or "validation"
    seed=42
)

labels = data.class_names
labels

Found 382 files belonging to 4 classes.
Using 306 files for training.


['Blotch_Apple', 'Normal_Apple', 'Rot_Apple', 'Scab_Apple']

In [28]:
def process(image, label):
    image = tf.cast(image / 255., tf.float32)
    return image, label

In [29]:
data_augmentation = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.1
)

In [30]:
resnet_model = keras.applications.ResNet50(
    include_top=False,  # Exclude the fully connected layer for now
    input_shape=(128, 128, 3),  # Input image size
)

In [31]:
num_classes = len(data.class_names)
top_layers = keras.Sequential([
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])


In [32]:
model = keras.Sequential([
    resnet_model,
    top_layers
])

In [33]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [34]:
history = model.fit(
    data,
    epochs=10,  # You can adjust the number of epochs
    validation_data=(data)  # Optional: Use augmented data for validation
)

Epoch 1/10
20/20 [==============================] - 71s 3s/step - loss: 1.5591 - accuracy: 0.5294 - val_loss: 3311.6252 - val_accuracy: 0.3072
Epoch 2/10
20/20 [==============================] - 51s 3s/step - loss: 1.0216 - accuracy: 0.6699 - val_loss: 864.2394 - val_accuracy: 0.3072
Epoch 3/10
20/20 [==============================] - 51s 3s/step - loss: 1.0113 - accuracy: 0.6667 - val_loss: 2646.3857 - val_accuracy: 0.3072
Epoch 4/10
20/20 [==============================] - 51s 3s/step - loss: 1.0421 - accuracy: 0.7190 - val_loss: 35736.4844 - val_accuracy: 0.3072
Epoch 5/10
20/20 [==============================] - 51s 3s/step - loss: 0.7823 - accuracy: 0.7353 - val_loss: 15627.7090 - val_accuracy: 0.2941
Epoch 6/10
20/20 [==============================] - 51s 3s/step - loss: 1.0274 - accuracy: 0.7353 - val_loss: 835.6677 - val_accuracy: 0.2941
Epoch 7/10
20/20 [==============================] - 51s 3s/step - loss: 0.9394 - accuracy: 0.7124 - val_loss: 28.0275 - val_accuracy: 0.1601
E

In [35]:
test_loss, test_acc = model.evaluate(data)
print("Test accuracy:", test_acc)

20/20 [==============================] - 9s 412ms/step - loss: 26.1880 - accuracy: 0.4216
Test accuracy: 0.4215686321258545


In [36]:
model.save("apple_disease_classification.keras")

In [37]:
test_data = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/apple-disease-detection/apple_disease_classification/Test',
    image_size=(128, 128),
    batch_size=16,
    validation_split=0.2,  # Split the dataset into training and validation
    subset="validation",    # Specify "validation" subset for testing
    seed=42
)



Found 120 files belonging to 4 classes.
Using 24 files for validation.


In [38]:
model.predict( test_data ) 

2/2 [==============================] - 2s 224ms/step


array([[1.6759867e-02, 3.4772748e-01, 4.8236182e-01, 1.5315077e-01],
       [2.9333773e-01, 2.6840800e-01, 2.9310355e-01, 1.4515075e-01],
       [0.0000000e+00, 0.0000000e+00, 9.9999994e-01, 0.0000000e+00],
       [6.0024798e-09, 9.9999815e-01, 1.5727909e-07, 1.6273963e-06],
       [1.7955925e-04, 9.9697328e-01, 1.7762023e-03, 1.0709312e-03],
       [4.3304652e-07, 9.9833375e-01, 1.9012716e-06, 1.6639188e-03],
       [5.6370421e-08, 9.5939785e-01, 4.0601965e-02, 2.6243060e-07],
       [0.0000000e+00, 0.0000000e+00, 9.9999994e-01, 0.0000000e+00],
       [1.2097616e-01, 2.3298146e-01, 6.2468088e-01, 2.1361459e-02],
       [1.2727360e-05, 6.5262133e-01, 3.4727192e-01, 9.3918861e-05],
       [2.3807163e-05, 8.8709050e-01, 1.1276388e-01, 1.2183310e-04],
       [0.0000000e+00, 0.0000000e+00, 9.9999994e-01, 0.0000000e+00],
       [5.2503677e-38, 6.1853973e-26, 9.9999994e-01, 6.8715919e-28],
       [1.2641937e-10, 9.9999982e-01, 5.2657317e-08, 1.4296663e-07],
       [0.0000000e+00, 0.0000000e+

In [39]:
import numpy as np

predictions = model.predict( test_data ) 

2/2 [==============================] - 1s 224ms/step


In [40]:
predicted_class_indices = np.argmax(predictions, axis = 1 )
len(predicted_class_indices)
for cls in predicted_class_indices : 
    print(cls) 

1
2
2
2
1
2
1
1
2
2
1
1
1
2
2
1
2
1
1
2
2
1
2
0


In [41]:
predicted_class_names = [data.class_names[idx] for idx in predicted_class_indices]
print((predicted_class_names))

['Normal_Apple', 'Rot_Apple', 'Rot_Apple', 'Rot_Apple', 'Normal_Apple', 'Rot_Apple', 'Normal_Apple', 'Normal_Apple', 'Rot_Apple', 'Rot_Apple', 'Normal_Apple', 'Normal_Apple', 'Normal_Apple', 'Rot_Apple', 'Rot_Apple', 'Normal_Apple', 'Rot_Apple', 'Normal_Apple', 'Normal_Apple', 'Rot_Apple', 'Rot_Apple', 'Normal_Apple', 'Rot_Apple', 'Blotch_Apple']


In [44]:
data = [
  # Row 3
    [1.6759867e-02, 3.4772748e-01, 4.8236182e-01, 1.5315077e-01],
       [2.9333773e-01, 2.6840800e-01, 2.9310355e-01, 1.4515075e-01],
       [0.0000000e+00, 0.0000000e+00, 9.9999994e-01, 0.0000000e+00],
       [6.0024798e-09, 9.9999815e-01, 1.5727909e-07, 1.6273963e-06],
       [1.7955925e-04, 9.9697328e-01, 1.7762023e-03, 1.0709312e-03],
]

# Label names and their corresponding column indices
labels = {
    "blotch": 0,
    "normal": 1,
    "rot": 2,
    "scab": 3,
}

# Function to find the column name with the minimum value
def find_min_label(row):
    min_index = min(range(len(row)), key=row.__getitem__)
    for label, index in labels.items():
        if index == min_index:
            return label

# Iterate through each row and find the column name with the minimum value
for i, row in enumerate(data):
    min_label = find_min_label(row)
    print(f"Image {i + 1}: Disease Name: '{min_label}'")


Image 1: Disease Name: 'blotch'
Image 2: Disease Name: 'scab'
Image 3: Disease Name: 'blotch'
Image 4: Disease Name: 'blotch'
Image 5: Disease Name: 'blotch'
